In [1]:
import numpy as np
import time

In [2]:
# Griewank function definition
def griewank(x):
    part1 = np.sum(x ** 2) / 4000
    part2 = np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1))))
    return part1 - part2 + 1


In [3]:
# Parameters
D = 30  # Dimension
NP = 100  # Population size
F = 0.5  # Scaling factor
CR = 0.9  # Crossover probability
lower_bound = -600
upper_bound = 600
max_generations = 1500
tol = 1e-5

In [4]:
# Seed for reproducibility
np.random.seed(42)

In [7]:
# Initialize population
pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([griewank(ind) for ind in pop])
best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]
fitness_history = [best_fitness]


In [8]:
# DE Main Loop
generation = 0
start_time = time.time()
while generation < max_generations and best_fitness > tol:
    generation += 1

    for i in range(NP):
        # Mutation
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, size=3, replace=False)
        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = np.clip(donor, lower_bound, upper_bound)

        # Crossover
        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            if np.random.rand() < CR or j == j_rand:
                trial[j] = donor[j]
            else:
                trial[j] = pop[i][j]

        # Selection
        f_trial = griewank(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()
    fitness_history.append(best_fitness)

    if generation % 50 == 0 or best_fitness <= tol:
        print(f"Generation {generation:3d}: Best Fitness = {best_fitness:.6e}")

    if best_fitness <= tol:
        print(f"\nTerminated at generation {generation} (fitness ≤ {tol}).")
        break


end_time = time.time()
elapsed_time = end_time - start_time


Generation  50: Best Fitness = 1.388632e+02
Generation 100: Best Fitness = 2.435374e+01
Generation 150: Best Fitness = 7.467420e+00
Generation 200: Best Fitness = 2.523706e+00
Generation 250: Best Fitness = 1.364189e+00
Generation 300: Best Fitness = 1.101710e+00
Generation 350: Best Fitness = 1.009415e+00
Generation 400: Best Fitness = 8.489633e-01
Generation 450: Best Fitness = 4.697617e-01
Generation 500: Best Fitness = 1.395148e-01
Generation 550: Best Fitness = 4.075925e-02
Generation 600: Best Fitness = 1.268398e-02
Generation 650: Best Fitness = 2.420582e-03
Generation 700: Best Fitness = 2.917687e-04
Generation 750: Best Fitness = 7.338552e-05
Generation 800: Best Fitness = 1.994331e-05
Generation 820: Best Fitness = 9.756707e-06

Terminated at generation 820 (fitness ≤ 1e-05).


In [9]:
# ------------------ Final Output ------------------
print("\nFinal Result:")
print(f"Best Vector: {best_vector}")
print(f"Best Fitness: {best_fitness}")
print(f"Total Time Taken: {end_time - start_time:.4f} seconds")


Final Result:
Best Vector: [-0.00107747 -0.00087314  0.0013931  -0.00174767  0.00136621  0.000119
  0.00133863 -0.00115537  0.00365855  0.00425153  0.00431182  0.00205955
 -0.00395195 -0.00031569  0.00042189 -0.00136612 -0.00268077 -0.00316953
 -0.00188811  0.00234167 -0.00806048 -0.00377977  0.00015583 -0.00034452
 -0.00371528  0.00487244 -0.00723454 -0.00151146 -0.00248067  0.0012582 ]
Best Fitness: 9.756706754404654e-06
Total Time Taken: 12.9845 seconds


In [10]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fitness_history,
    x=list(range(len(fitness_history))),
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue'),
    marker=dict(size=4)
))

fig.update_layout(
    title='Differential Evolution on levy Function',
    xaxis_title='Generation',
    yaxis_title='Fitness Value',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
